In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy import stats
import math

In [ ]:
def write_eccentricity(s):
    values = [7, 18]
    conditions = [(s['standard_pos'] == -2),
                  (s['standard_pos'] == 9)]
    s['eccentricity'] = np.select(conditions, values)

In [ ]:

sub_list = ["sub-01","sub-02","sub-03",
            "sub-04","sub-05","sub-06",
            "sub-07","sub-08","sub-09",
            "sub-10","sub-11","sub-12",
            "sub-13","sub-15","sub-16",
            "sub-17","sub-18","sub-19",
            "sub-21","sub-22","sub-23",
            "sub-24","sub-25","sub-26",
            "sub-27","sub-28","sub-30",
            "sub-31","sub-32","sub-33",]


run_list = ["run-01","run-02","run-03","run-04"]

# Path to the folder containing subject-specific subfolders
pth = "/home/lisz/Desktop/Time_project/time_fmri/behavioral_data/raw_behavioral_data"
pth_save = "/home/lisz/Desktop/Time_project/time_fmri/behavioral_data"

In [ ]:
'''
open raw file
set default of exclusion as "stay"
mark values below 0.5 and above 8 seconds rt as "exclude"
if button was pressed before the second stimuli appearance - also "exclude"
use function which writes eccentricity
save to a folder "behavioral_data"
DO NOT DELETE TRIALS, JUST MARK THEM
'''

for sub in sub_list:    
    for run in run_list:
        df = pd.read_csv(pth+f"/{sub}/{sub}_{run}_Reproduction_exp_timeline.csv")
        df['exclusion'] = "stay"
        for index, value in df['key_resp_rt'].items():
            if value < 0.5 or value > 8 or math.isnan(value):
                df.at[index, 'exclusion'] = "exclude"
        if "press_error.rt" in df.columns:
            for index, value in df['press_error.rt'].items():
                if value > 0:
                    df.at[index, 'exclusion'] = "exclude"
        #print("Failed trials",sub,run,len(df[df['exclusion']=='exclude']))
        write_eccentricity(df)
        
        file_dir = os.path.join(pth_save, sub)
        os.makedirs(file_dir, exist_ok=True)
        df.to_csv(f'{file_dir}/{sub}_{run}.csv', sep=',',index=False)

#### Delete answers where the difference for 1 or 2 answers is bigger than 1.5 seconds

no more than 2 responses within the subject and had a difference of 1.5 seconds or more from the other responses

In [ ]:
'''
open files direction where already filtered data is stored
in column "exclusion" take row with index_n and change value to 'exclude'
save back to the folder with filtered data
'''

def manual_exclusion(sub,run,index_n):
    file_dir = os.path.join(pth_save, sub)
    os.makedirs(file_dir, exist_ok=True)
    df = pd.read_csv(f'{file_dir}/{sub}_{run}.csv')
    # in column "exclusion" take row with index_n and change value to 'exclude'
    df.at[index_n, 'exclusion'] = "exclude"
    print("Exclued trials, index",index_n,sub,run)
    
    df.to_csv(f'{file_dir}/{sub}_{run}.csv', sep=',',index=False)
    #return df

In [ ]:
manual_exclusion('sub-04','run-02',20)
manual_exclusion('sub-10','run-04',17)

manual_exclusion('sub-16','run-02',3)
manual_exclusion('sub-19','run-03',17)

manual_exclusion('sub-24','run-04',14)
manual_exclusion('sub-26','run-03',17)

manual_exclusion('sub-31','run-04',7)

### Create df for each subject for all runs

In [ ]:
'''
concat runs for each subject and save it
'''

for sub in sub_list:
    data1 = pd.read_csv(pth_save+f"/{sub}/{sub}_run-01.csv")
    data2 = pd.read_csv(pth_save+f"/{sub}/{sub}_run-02.csv")
    data3 = pd.read_csv(pth_save+f"/{sub}/{sub}_run-03.csv")
    data4 = pd.read_csv(pth_save+f"/{sub}/{sub}_run-04.csv")
    data_list = [data1,data2,data3,data4]
    data_all_runs = pd.concat(data_list)
    data_all_runs.to_csv(pth_save+f'/{sub}/{sub}_all_runs.csv', sep=',')

In [ ]:
'''
open each subjects data into one dataset
'''

df_list = []
for sub in sub_list:    
    df = pd.read_csv(pth_save+f"/{sub}/{sub}_all_runs.csv")
    df_list.append(df)
    
df_all = pd.concat(df_list)
df_all = df_all.dropna(subset=['key_resp_rt'])

In [ ]:
"""
SUBLIST WITHOUT THOSE WHO HAVE BLINDSPOT AROUND 18 degrees
  

sub_list = [ sub1 , sub2 , sub3 ,
             sub5 , sub6 ,
             sub8 ,
             sub10 , sub11 ,
             sub15 , sub16 ,
             sub18 , sub19 ,
             sub22 , sub23 ,
             sub24 , sub26 ,
             sub27 , sub28 , sub30 ,
             sub31 , sub32 ]""" 

In [ ]:
print(df_all['participant'].unique())

In [ ]:
'''
looking at behavioral data, drop those which are marked
as "excluded"
'''

df_all = df_all[df_all['exclusion'] != 'exclude']

In [ ]:
plt.figure(figsize=(15, 15))
g = sns.catplot(y="key_resp_rt", 
               data = df_all,
               col="participant", col_wrap=6, kind="point",
               height=4,
               errwidth=2, capsize=0.1,
               dodge=True,sharex=False)
for ax in g.axes.flat:
    ax.axhline(2, ls='--', color='silver')

In [ ]:
g = sns.FacetGrid(df_all, col="participant", col_wrap=6, height=4, sharex=False)

# Use map to create histograms for each participant
g.map(sns.histplot, "key_resp_rt")

In [ ]:
df_all['key_resp_rt'].describe()[['mean','std']]

In [ ]:
mean_rt = df_all.groupby("participant")["key_resp_rt"].mean().reset_index()
mean_rt

In [ ]:
sns.scatterplot(x=mean_rt['participant'], y=mean_rt['key_resp_rt'], color='darkred')
plt.xticks([])

# Rename y-axis
plt.ylabel('Reproduced duration (s)')
plt.xlabel('Subjects')

# Add a title
#plt.title('Mean reproduced duration by each participant')

# Plot a horizontal line at y=2
plt.axhline(y=2, color='gray', linestyle='--', label='stimulus duration')

# Display legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
sns.histplot(data=mean_rt, bins=30)

In [ ]:
'''
create two lists, subjects who overestimate and underestimate
'''
df_above = mean_rt[mean_rt['key_resp_rt'] > 2]
df_below = mean_rt[mean_rt['key_resp_rt'] < 2]
print(df_above['participant'].unique())
print(len(df_above['participant'].unique()))

In [ ]:
print(df_below['participant'].unique())
print(len(df_below['participant'].unique()))

In [ ]:
plt.style.use('default') # style of the background
fig = plt.figure()
ax = plt.axes()
sns.lineplot(x='eccentricity', 
            y='key_resp_rt', 
            data=df_all, 
            hue = 'vis_area',
            ci=68,#standard error or mean
            err_style='bars',
            palette = dict(perifoveal="steelblue", peripheral="burlywood"),
            err_kws = {'capsize':2, 
                       'elinewidth':1.5, # errorbar width
                       'capthick':1.5})
#plt.axhline(2, ls='--', color='silver')
plt.ylim(1.8, 2.4)
plt.xlabel('Eccentricity')
plt.ylabel('Response time')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks([7,18])

#plt.text(0.2,2.01, "standard duration", color='silver')
plt.title('fMRI exp, N=30')
plt.legend(title = "Eccentricities", prop={'size': 7}, 
           loc='lower right',labels=['perifoveal (-7°,7°)', 'peripheral (-21°,21°)'])

## Confirm that eccentricity is not significant

In [ ]:
# isi in seconds
res = pg.rm_anova(dv='key_resp_rt', within=['eccentricity'], 
                  subject='participant', 
                  data=df_all, detailed=True, effsize='np2')
res

### Normality check

In [ ]:
s1 = dt[dt["participant"]==1]
s2 = dt[dt["participant"]==2]
s3 = dt[dt["participant"]==3]

s4 = dt[dt["participant"]==4]
s5 = dt[dt["participant"]==5]
s6 = dt[dt["participant"]==6]

s7 = dt[dt["participant"]==7]
s8 = dt[dt["participant"]==8]
s9 = dt[dt["participant"]==9]

s10 = dt[dt["participant"]==10]
s11 = dt[dt["participant"]==11]
s12 = dt[dt["participant"]==12]

s13 = dt[dt["participant"]==13]
s14 = dt[dt["participant"]==14]
s15 = dt[dt["participant"]==15]

s16 = dt[dt["participant"]==16]

In [ ]:
all_subj= [s1,s2,s3,
           s4,s5,s6,
           s7,s8,s9,
           s10,s11,s12,
           s13,s14,s15,
           s16]

In [ ]:
def normality_check():
    list_means = []
    for i in subject_list:
        a = i.loc[(i['eccentricity']==18)]
        b = a.key_resp_rt.mean()
        list_means.append(b)
    return list_means
all_sub_means = normality_check()

In [ ]:
pg.normality(all_sub_means, dv='key_resp_rt', 
             group='participant',
             method='shapiro')

### Normality check done, all normal

In [ ]:
res = stats.probplot(all_sub_means, plot=plt)
plt.title('pause duration = 2s, eccentricity = -21')
plt.show()

## Debug

In [ ]:
a = s16.loc[(s16['key_resp_rt']>2.5)]
a